# Project objectives 

1. Protect clients data of insurance company to prevent data leaks. 
2. Create algorythms of encoding data so that data passed to 3rd parties cannot be interpreted. 
3. Data encoding process must not decresce ML quality.
4. There must be a way to decode data back. 

## 1. Loading data

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score,r2_score

In [6]:
data = pd.read_csv('insurance.csv')

In [7]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.tail()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [8]:
print('')
print('== Duplicates ==')
print('')
data.duplicated().sum()

duplicates = data[data.duplicated() == True]
print(len(duplicates))


== Duplicates ==

153


### Stage summary:
1. data loaded and analyzed
2. dicovered duplicates


<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='green'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
Данные на базе) ОК, хорошо, когда есть аналитическая культура - обязательно делать первичный анализ данных, не смотря на проект)

## 2. Matrix multiplication


Symbols:

- $X$ — feature matrix (0 column consist of 1s)

- $y$ — target vector

- $P$ — matrix used to multiple features

- $w$ — vector of regression weigths

Predictions:

$$a = Xw$$

Learning goal:

$$
w = \arg\min_w MSE(Xw, y)
$$

Learning formula:

$$
w = (X^T X)^{-1} X^T y
$$

Encoding folmula explanation:

$$
a = XP((XP)^T XP)^{-1} (XP)^T y
$$

Open left brackets with transposition:

$$
a= XP((P^T(X^TX))P)^{-1} (XP)^T y
$$

Open brackets with power of "-1" for the first element of multiplication (we have 3 elements uder power of "-1"):

$$
a= XPP^{-1}(P^T(X^TX))^{-1}) (XP)^T y
$$

Open the rest under power "-1" and transposition:

$$
a= XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^T X^T y
$$

Optimize cos we have 2 E matrices:

$$
a= XE(X^TX)^{-1}E X^T y
$$


$$
a= X(X^TX)^{-1} X^T y
$$


## 3. Algorythm description

**Algorythm**

Multiply features on revesed matrix, generated with normal condition and applicable size

**Explanation**

In the section above we have mathematically proved that multiplication by matirix will not change the output.  However, we haven't considetred neccessity:
1) noramlly distributed features 
2) sizes of matrices

Thereafter, we will use np.random.normal to generate matrix for multiplication, in which we set appropriate matrix size.

Additionally this matrix can be inverted, meaning is has inverse matrix, which can be used to decode data. 

## 4. Algorythm probation

Select features and target

In [8]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

Split data to train and test

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              train_size=0.75, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

Train model on original data, calculate metric r2 and store it for future comparison on metric calculated on encoded data.

In [10]:
model = LinearRegression()
model.fit(features_train,target_train)
predicted = model.predict(features_test) 
r2_real = r2_score(target_test, predicted)

Generate matrix to encode data

In [11]:
mu, sigma = 5, 2
np.random.seed(0)
matrix = np.random.normal(mu, sigma, size=(4, 4))

In [12]:
matrix

array([[8.52810469, 5.80031442, 6.95747597, 9.4817864 ],
       [8.73511598, 3.04544424, 6.90017684, 4.69728558],
       [4.7935623 , 5.821197  , 5.28808714, 7.90854701],
       [6.52207545, 5.24335003, 5.88772647, 5.66734865]])

Invert generated matrix

In [13]:
matrix_reversed = np.linalg.inv(matrix)

In [14]:
matrix_reversed

array([[ 0.60836868, -0.38976102, -0.87002694,  0.51929855],
       [ 0.09316757, -0.48843299, -0.4290607 ,  0.84769108],
       [-0.94263732,  0.82292682,  1.1576445 , -0.72042903],
       [ 0.19297456,  0.04550769,  0.19554163, -0.45699611]])

Test whther we get E matrix when multiply originally generated matrix with inverted version of it

In [15]:
np.dot(matrix, matrix_reversed)

array([[ 1.00000000e+00, -6.50325773e-16,  8.26379268e-16,
        -1.53611981e-15],
       [-6.56835333e-16,  1.00000000e+00,  5.02563939e-18,
        -2.00604225e-16],
       [-4.05471270e-16, -4.24812667e-16,  1.00000000e+00,
        -5.98455754e-16],
       [-1.36537189e-15, -7.75096634e-17,  1.31340658e-15,
         1.00000000e+00]])

Diagonal has 1's, other elements has numbers close to 0, which is acceptable because we are working with float. 

Modify fearutes matrix by matrix multiplication with generated matrix


In [16]:
train = np.dot(features_train,matrix)

Check out results

In [17]:
train

array([[173917.61529744, 210869.32930712, 191738.30735839,
        286506.53431926],
       [276414.7103214 , 335410.29284189, 304835.38288609,
        455701.49749837],
       [197301.45616939, 239353.89442506, 217567.07493461,
        325197.26275968],
       ...,
       [214645.42458499, 260343.41295159, 236673.24771445,
        353719.78936646],
       [240375.73190365, 291729.94306764, 265108.52062655,
        396344.21507514],
       [195743.30889727, 237562.70119874, 215885.05876659,
        322757.96659426]])

This data is definitely different from original features matrix. No one can guess what kind of data is stored, i.e where is gender, age, salary, and so on. 

Perform similar transformation of test data

In [18]:
test = np.dot(features_test, matrix)

Train model on encoded data and calculate metric.


In [19]:
model = LinearRegression()
model.fit(train,target_train)
predicted = model.predict(test) 
r2_masked = r2_score(target_test, predicted)

Now let's find the difference between r2 calculated on original data and r2 on encoded. 

In [20]:
r2_real- r2_masked

-2.1949109196839345e-13

2,4 * 10 ^ (-12), difference close to 0. We can infer, that out encoding approach has no impact of quality of model. 

Let's perform decoding:

In [21]:
train_2 = train.copy()

In [22]:
pd.options.display.float_format = '{:,.10f}'.format

In [23]:
data3 = np.dot(train_2,matrix_reversed)

In [24]:
pd.DataFrame(data3)

,0,1,2,3
0,1.0000000000,43.0000000000,"36,200.0000000000",1.0000000000
1,1.0000000000,34.0000000000,"57,600.0000000000",-0.0000000000
2,-0.0000000000,32.0000000000,"41,100.0000000001",1.0000000000
3,-0.0000000000,36.0000000000,"45,100.0000000000",1.0000000000
4,-0.0000000000,33.0000000000,"50,600.0000000000",2.0000000000
...,...,...,...,...
3745,-0.0000000000,42.0000000000,"32,100.0000000000",-0.0000000000
3746,-0.0000000000,28.0000000000,"22,700.0000000000",4.0000000000
3747,0.9999999999,41.0000000000,"44,700.0000000001",0.9999999999
3748,0.0000000000,22.0000000000,"50,100.0000000000",4.0000000000


Decoding output gives us original values with very minor deviation.

## Project summary:
1. Algorythm of data encoding and decoding was proposed to meet the goal of the project.
2. To verify algorythm it was tested on real data with regression model.
3. The models quality metrics idfference (r2_real- r2_masked) is close to zero: -2.1949109196839345e-13, which proves that our encoding approach has no impact on traning the model. 
4. Decoding of the data was done successfuly. 